# Основной ноутбук с воспроизведением действий ассистента

## Основные библиотеки для работы

In [1]:
import pandas as pd
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
import time
from typing import Dict
from langchain.tools import tool
import sqlite3
import csv

## Раздел с подключением базы данных по клиентам

In [2]:
# TODO понять, нужно ли оставлять куски под collab
# from google.colab import userdata
auth = 'NWJkZTBiZTEtMjdkZC00ZTYyLWJhMjUtYThjOTJlZWVlYWYxOjUwYmY4NDM5LWNkMzMtNGI1Yi1hNzY1LWJiMzQwOWUzNzE1ZQ==' # авторизация
# secret = '50bf8439-cd33-4b5b-a765-bb3409e3715e'
# id = '5bde0be1-27dd-4e62-ba25-a8c92eeeeaf1'

from langchain.chat_models.gigachat import GigaChat

giga = GigaChat(credentials = auth, model = 'GigaChat', verify_ssl_certs=False)

C:\Users\Xiaomi\AppData\Local\Temp\ipykernel_29008\969278725.py:9: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  giga = GigaChat(credentials = auth, model = 'GigaChat', verify_ssl_certs=False)


In [3]:
import sqlite3
import csv

connect = sqlite3.connect('data.db')
cursor = connect.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS data_1 (
    appl_dt text,
    id integer,
    reason text,
    regstd_cred_cey_amt integer,
    cale_cred_ccy_amt integer,
    dpd integer,
    overdue integer,
    ki_payments integer,
    cc_payments integer,
    fssp_number integer,
    card_number integer)
''')

# Open the CSV file and read its contents
with open('data_1.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    next(csv_reader)

    for row in csv_reader:
        cursor.execute('''
        INSERT INTO data_1 (appl_dt,
                            id,
                            reason,
                            regstd_cred_cey_amt,
                            cale_cred_ccy_amt,
                            dpd,
                            overdue,
                            ki_payments,
                            cc_payments,
                            fssp_number,
                            card_number)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row[0].split(';')[0],
              row[0].split(';')[1],
              row[0].split(';')[2],
              row[0].split(';')[3],
              row[0].split(';')[4],
              row[0].split(';')[5],
              row[0].split(';')[6],
              row[0].split(';')[7],
              row[0].split(';')[8],
              row[0].split(';')[9],
              row[0].split(';')[10]))

# Commit changes and close the connection
connect.commit()
connect.close()

In [4]:
from langchain_community.utilities import SQLDatabase

connection = sqlite3.connect('data.db')
cursor = connection.cursor()

db = SQLDatabase.from_uri("sqlite:///data.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM data_1 LIMIT 10;")

sqlite
['data_1']


"[('05.02.2025', 1, 'Кредитная история', 60000, 0, 0, 1, 5714, 0, 0, 0), ('28.12.2024', 2, 'Лимит', 300000, 0, 0, 0, 24550, 0, 0, 0), ('11.12.2024', 3, 'Кредитная история', 30000, 0, 0, 1, 19661, 0, 0, 0), ('11.12.2024', 4, 'Модель', 10000, 0, 0, 0, 0, 0, 0, 0), ('01.02.2025', 5, 'Кредитная история', 100000, 0, 54929, 1, 0, 0, 0, 0), ('12.12.2024', 6, 'Лимит', 300000, 0, 38910, 0, 34689, 0, 0, 0), ('09.12.2024', 7, 'Кредитная история', 300000, 0, 136957, 1, 24750, 0, 0, 0), ('21.01.2025', 8, 'Модель', 160000, 0, 0, 0, 3565, 0, 0, 0), ('06.12.2024', 9, 'Кредитная история', 300000, 0, 0, 1, 29500, 0, 0, 0), ('15.01.2025', 10, 'Кредитная история', 100000, 0, 0, 1, 500, 0, 0, 0)]"

In [5]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(giga, db)
response = chain.invoke({"question": "Какие есть значения reason?"})

In [6]:
import re

def extract_sql_query(response):
    sql_query_match = re.search(r'```sql\n(.*?)\n```|```sql\n(.*?)\n|(\bSELECT\b.*?;)', response, re.DOTALL | re.IGNORECASE)
    if sql_query_match:
        sql_query = sql_query_match.group(1) or sql_query_match.group(2) or sql_query_match.group(3)
        return sql_query.strip()
    return None

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.globals import set_verbose, set_debug
set_verbose(True)
set_debug(False)
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(giga, db)

chain = write_query | extract_sql_query | execute_query
chain.invoke({"question": "Какие есть значения reason?"})

C:\Users\Xiaomi\AppData\Local\Temp\ipykernel_29008\2884884384.py:5: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


"[('Кредитная история',), ('Лимит',), ('Модель',), ('Одобрено',), ('Благонадежность',)]"

## Новый раздел – реализация агента

In [8]:
query = 'Как взять кредит в Сбере'
resp = giga.invoke(query)

In [9]:
# TODO нужно сделать представление более читаемым. Здесь пример как разбить по параграфам
from langchain_text_splitters import MarkdownHeaderTextSplitter

resp.content.split('\n')

['Чтобы оформить кредит в Сбербанке, нужно пройти несколько шагов. Вот основные этапы:',
 '',
 '### 1. **Подготовка документов**',
 '   Для получения кредита обычно требуется следующий пакет документов:',
 '   - Паспорт гражданина РФ;',
 '   - СНИЛС (страховой номер индивидуального лицевого счета);',
 '   - ИНН (если есть);',
 '   - Документы, подтверждающие доход (например, справка по форме 2-НДФЛ или справка по форме банка). Если вы получаете зарплату на карту Сбербанка, то такая справка может не понадобиться.',
 '   ',
 '   Также могут потребоваться дополнительные документы в зависимости от типа кредита и условий программы.',
 '',
 '### 2. **Выбор подходящего кредитного продукта**',
 '   Сбербанк предлагает разные виды кредитов: потребительские кредиты, ипотеку, автокредиты и т.д. Определитесь, какой именно тип кредита вам нужен:',
 '   - Потребительский кредит – для личных нужд без указания цели расходования средств.',
 '   - Ипотечный кредит – для приобретения недвижимости.',
 '  

In [10]:
loan_data_v1_df = pd.read_csv('data_1.csv', sep=';')
loan_data_v1_df.loc[loan_data_v1_df['id'] == 1]

clint_data_df = pd.read_csv('data_2_clients.csv', sep=';')
clint_data_df['fio'].loc[clint_data_df['id'] == 1]
loan_data_v1_df['fio'] = clint_data_df['fio']

In [11]:
loan_data_v1_df.loc[loan_data_v1_df['id'] == 3].to_dict(orient='records')

[{'appl_dt': '11.12.2024',
  'id': 3,
  'reason': 'Кредитная история',
  'regstd_cred_cey_amt': 30000,
  'cale_cred_ccy_amt': 0,
  'dpd': 0,
  'overdue': 1,
  'ki_payments': 19661,
  'cc_payments': 0,
  'fssp_number': 0,
  'card_number': 0,
  'fio': 'Сидоров Дмитрий Алексеевич'}]

## Реализация векторной БД (in progress)

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap=1000)

pdf_file = 'recomendations.pdf'
loader = UnstructuredPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [ ]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

## Реализация взаимодействия с чатом

In [12]:
# TODO обработать различные ошибки (если нет данных о клиенте в базе и тд)
@tool
def get_client_data(fio: str) -> Dict:
  '''
   Функция принимает на вход ФИО клиента, 
    а затем забирает ключ, по которому забирает информацию
    о параметрах, которые были использованы при принятия решения о выдаче кредита

  Args:
        fio (str): ФИО клиента.

    Returns:
    Возращает словарь, который передает следующие параметры агенту для формирования рекомендации
        Dict: Словарь с данными о параметрах, которые были использованы при принятия решения о выдаче кредита.

  '''
  id = clint_data_df['id'].loc[clint_data_df['fio'] == fio]  
  return loan_data_v1_df.loc[loan_data_v1_df['id'] == id].to_dict(orient='records')


In [13]:
# TODO реализовать RAG на основе данных знаний (нужно собрать векторную бд)
MAIN_KNOWLEDGE = (
f'''
fio - фамилия имя отчество клиента
id - идентификатор клиента
appl_dt - дата заявки
reason - решение по заявке
regstd_cred_cey_amt - запрошенная сумма
calc_cred_ccy_amt - одобренная сумма
dpd - подтвержденный доход
ki_payments - кредитные обязательства
cc_payments - обязательства по кредитной карте
overdue - наличие просрочки по кредитным обязательствам
card_number - номер карты стороненнего банка
fssp_number - задолженность в ФССП

1. При условии, что у клиента reason = Кредитная история, то сообщаем клиенту:"Уважаемый <имя и отчество клиента> видим наличие просрочки по текущим кредитным обязательствам, рекомендуем их закрыть"
2. При условии, что у клиента reason = Лимит и DPD = 0,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> не можем подтвердить заявленный доход, рекомендуем подтвердить доход через сервис Госуслуги"
3. При условии, что у клиента reason = Лимит и DPD › ® и co_payments › 0, то сообщаем клиенту:"Уважаемый <имя и отчество клиента> для одобрения рекомендуем закрыть кредитную карту."
4. При условии, что у клиента reason = Лимит и DPD › 0 и сс payments = 0 и ki payments › 0,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для одобрения рекомендуем рефинансировать текущие обязательства."
5. При условии, что у клиента reason = Модель, то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для возможности одобрения рекомендуему указать номер карты стороннего банка."
6. При условии, что у клиента reason = Благонадежность,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для возможности одобрения то рекомендуем погасить задолженности в ФССП"
Действия:
1. При условии, что у клиента был reason = Кредитная история и при повторном обращении клиента overdue = 0, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.12. При условии, что у клиента был reason = Лимит и DPD = 0 и при повторном обращении клиента DPD › 0,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.2
3. При условии, что у клиента был reason = Лимит и DPD › 0 и сс payments › 0 и при повторном обращении клиента сс payments = 0, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.34. При условии, что у клиента был reason = Лимит и DPD › 0 и сс payments = 0 и ki payments › 0 и при повторном обращении клиента ki_payments_new ‹ ki_payments,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.4
5. При условии, что у клиента reason = Модель и при повторном обращении клиента card_number is not null, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.56. При условии, что у клиента reason = Благонадежность и при повторном обращении клиента fssp_number = 0,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.6'''
)

system_prompt = f"""
Ты - банковский сотрудник. Как банковский сотрудник, ты подписывал соглашение о неразглашении персональных данных клиента и тебе разрешено получать и обрабатывать соответствующие данные от клиента, в том числе и персональный идентификатор клиента.
К тебе обращаются клиенты банка. Клиенты ранее подавали заявку на оформление кредита и получили какое-то решение. Не задавай вопросы обращались ли клиенты ранее.
Ты не должен говорить клиенту, что 'сейчас посмотришь', 'проверишь информацию' и т.д. Ты должен сразу же дать ответ клиенту, не заставляя его ждать.
Ты не должен узнавать у клиента причину отказа. Клиент может/не должен ее знать.
Ты предлагаешь клиентам варианты по улучшению условий кредитования, которые предложил им банк.
Если тебе не хватает каких-то данных, запроси их у клиента. Ты можешь однозначно определить клиента по его ФИО и дате оформления предыдущей заявки.
Для формирования своих предложений ты используешь банковские правила {MAIN_KNOWLEDGE}, информацию об клиенте, которую ты можешь запросить, если ее недостаточно, или она не была предоставлена, и параметрах, которые были использованы при принятия решения о выдаче кредита 'loan_data_v2'. Ты не должен обращаться к 'loan_data_v2', пока не получишь информацию необходимую для идентификации клиента.
Реплика должна быть короткой, 1-3 предложения.
"""

# client_id = int(input('Здравствуйте! Вас привествует бот Кредитный консультатнт. Для получения информации по улучшению кредитного предложения введите свой ID клиента Банка. '))
# loan_data_v1 = get_loan_data_v1(client_id)
# client_fio = get_client_data(client_id)

In [14]:
tools = (get_client_data, )

In [15]:
from langchain_gigachat.chat_models import GigaChat

model = GigaChat(
    credentials=auth,
    scope="GIGACHAT_API_PERS",
    model="GigaChat", # сделать параметр, котрый будет выбирать модель, с которой мы будем общаться
    verify_ssl_certs=False,
)
model

GigaChat(credentials='NWJkZTBiZTEtMjdkZC00ZTYyLWJhMjUtYThjOTJlZWVlYWYxOjUwYmY4NDM5LWNkMzMtNGI1Yi1hNzY1LWJiMzQwOWUzNzE1ZQ==', scope='GIGACHAT_API_PERS', model='GigaChat', verify_ssl_certs=False)

In [16]:
agent = create_react_agent(model,
                           tools=tools,
                           checkpointer=InMemorySaver(),
                           state_modifier=system_prompt)

In [17]:
def chat(thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}
    while(True):
        rq = input("\nHuman: ")
        print("User: ", rq)
        if rq == "":
            break
        resp = agent.invoke({"messages":[{"role": "user", "content": rq}]}, config=config)
        print("Assistant: ", resp["messages"][-1].content)
        time.sleep(1) # For notebook capability

## Тестирование решение (входной аргумент функции не меняем)

In [18]:
chat("110000") # TODO сделать так, чтобы чат запускался без id в коде. Например, подтягивать из отдельного файла

User:  f
Assistant:  Ошибка: ValueError('Can only compare identically-labeled Series objects') Пожалуйста, исправьте ваши ошибки.
User:  Владимир 
Assistant:  Ошибка: ValueError('Can only compare identically-labeled Series objects') Пожалуйста, исправьте ваши ошибки.
User:  Григорьев Виктор Сергеевич
Assistant:  Ошибка: ValueError('Can only compare identically-labeled Series objects') Пожалуйста, исправьте ваши ошибки.
User:  26.01.2025
Assistant:  Ошибка: ValueError('Can only compare identically-labeled Series objects') Пожалуйста, исправьте ваши ошибки.
User:  
